<a href="https://colab.research.google.com/github/Esbern/Python-for-Planners/blob/main/class_random_walk-coffishop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import folium
import random
import math
from pyproj import Transformer

In [9]:
import folium
import random
import time
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Transformer
from IPython.display import display, clear_output

# ---- Coordinate Transformers (WGS84 <-> EPSG:25832) ----
wgs_utm = Transformer.from_crs("EPSG:4326", "EPSG:25832", always_xy=True)
utm_wgs = Transformer.from_crs("EPSG:25832", "EPSG:4326", always_xy=True)

# ---- Constants ----
STEP_SIZE = 100  # Max step size (~20 meters)
PAUSE_TIME = 3  # Time steps to pause at a coffee shop
COFFEE_RADIUS = 50  # Distance to stop at a coffee shop (meters)

# ---- Coffee Shop Class ----
class CoffeeShop:
    def __init__(self, lon, lat):
        """Stores coffee shop location in UTM (EPSG:25832)."""
        x, y = wgs_utm.transform(lon, lat)  # Convert to UTM
        self.location = Point(x, y)

    def get_marker(self):
        """Returns a Folium marker for this coffee shop."""
        lon, lat = utm_wgs.transform(self.location.x, self.location.y)  # Convert back to WGS84
        return folium.Marker([lat, lon], popup="Coffee Shop", icon=folium.Icon(color="blue"))

# ---- Walker Class ----
class Walker:
    def __init__(self, lon, lat):
        """Initializes walker location in UTM (EPSG:25832)."""
        x, y = wgs_utm.transform(lon, lat)  # Convert to UTM
        self.location = Point(x, y)
        self.pause_counter = 0  # Tracks pause duration
        self.color = "red"  # Default color

    def move(self, coffee_shops):
        """Moves walker randomly unless paused. Stops at coffee shop if nearby."""
        if self.pause_counter > 0:
            self.pause_counter -= 1  # Continue waiting
        else:
            # Generate a random movement in UTM (meters)
            dx = (random.random() - 0.5) * STEP_SIZE
            dy = (random.random() - 0.5) * STEP_SIZE
            new_pos = Point(self.location.x + dx, self.location.y + dy)

            # Check if near a coffee shop
            for shop in coffee_shops:
                if new_pos.distance(shop.location) < COFFEE_RADIUS:
                    self.pause_counter = PAUSE_TIME  # Stop for PAUSE_TIME steps
                    new_pos = shop.location  # Snap to coffee shop
                    self.color = "green"  # Change color to green
                    break
                else:
                  self.color = "red"  # Reset color to red

            self.location = new_pos  # Update position

    def get_marker(self):
        """Returns a Folium marker for this walker."""
        lon, lat = utm_wgs.transform(self.location.x, self.location.y)  # Convert back to WGS84
        return folium.Marker([lat, lon], popup="Walker", icon=folium.Icon(color=self.color))

# ---- Initialize Objects ----
coffee_shops = [CoffeeShop(12.5690, 55.6770), CoffeeShop(12.5700, 55.6760)]
walkers = [Walker(12.5683, 55.6761) for _ in range(5)]  # 5 random walkers

# ---- Run Simulation ----
for step in range(30):  # 30 iterations
    clear_output(wait=True)

    # Move all walkers
    for walker in walkers:
        walker.move(coffee_shops)

    # Create a new map
    m = folium.Map(location=[55.6761, 12.5683], zoom_start=15)

    # Add coffee shops to map
    for shop in coffee_shops:
        m.add_child(shop.get_marker())

    # Add walkers to map
    for walker in walkers:
        m.add_child(walker.get_marker())

    # Display map
    display(m)
    time.sleep(0.5)  # Pause before next iteration
